In [1]:
import json
import pandas as pd
import numpy as np
import csv
from faker import Faker
from faker.providers import internet
import datetime
from sqlalchemy import create_engine
import psycopg2
from config import db_password
import time

In [2]:
zillow_df = pd.read_csv('Resources/Zillow_Sale_Prices_Zip.csv')
zillow_df.set_index('RegionName', inplace=True)
zillow_df.head()

,RegionID,StateName,SizeRank,2008-03,2008-04,2008-05,2008-06,2008-07,2008-08,2008-09,...,2019-05,2019-06,2019-07,2019-08,2019-09,2019-10,2019-11,2019-12,2020-01,2020-02
RegionName,,,,,,,,,,,,,,,,,,,,,
35004,73257,Alabama,5932,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,168800.0,167900.0,169400.0,174700.0,180300.0,181200.0,184100.0,189800.0,186400.0,183500.0
35005,73258,Alabama,6326,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,92100.0,97200.0,101800.0,92000.0,89200.0,90700.0,90700.0,95500.0,108100.0,NaN
35007,73260,Alabama,3559,145700.0,145700.0,152500.0,148600.0,146500.0,139600.0,140200.0,...,175200.0,180300.0,178100.0,180600.0,185900.0,191500.0,194100.0,191900.0,193200.0,186000.0
35022,73270,Alabama,4006,177400.0,178900.0,163900.0,157000.0,158600.0,163700.0,164400.0,...,190800.0,195600.0,184700.0,183400.0,183200.0,201700.0,198900.0,214400.0,210500.0,226500.0
35023,73271,Alabama,3755,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,106700.0,111400.0,103000.0,90000.0,87200.0,93700.0,99900.0,101100.0,108000.0,NaN


In [3]:
zillow_df.dtypes

RegionID       int64
StateName     object
SizeRank       int64
2008-03      float64
2008-04      float64
              ...   
2019-10      float64
2019-11      float64
2019-12      float64
2020-01      float64
2020-02      float64
Length: 147, dtype: object

In [4]:
california_df = zillow_df.loc[zillow_df.StateName=='California',:]
california_df.head()

,RegionID,StateName,SizeRank,2008-03,2008-04,2008-05,2008-06,2008-07,2008-08,2008-09,...,2019-05,2019-06,2019-07,2019-08,2019-09,2019-10,2019-11,2019-12,2020-01,2020-02
RegionName,,,,,,,,,,,,,,,,,,,,,
90002,95983,California,2748,292100.0,296700.0,268800.0,238900.0,203200.0,202100.0,172800.0,...,379500.0,385100.0,388900.0,381400.0,387800.0,400800.0,408800.0,393500.0,378600.0,387600.0
90003,95984,California,1036,NaN,NaN,NaN,306000.0,260500.0,248900.0,220500.0,...,397600.0,417400.0,436800.0,436800.0,418300.0,395900.0,374500.0,394200.0,397500.0,424800.0
90008,95989,California,1754,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,801700.0,850800.0,875000.0,935000.0,893600.0,834300.0,816100.0,859000.0,868500.0,883500.0
90016,95997,California,1060,462900.0,416000.0,347700.0,347700.0,337300.0,339800.0,320800.0,...,714100.0,737500.0,734900.0,733300.0,723000.0,760400.0,766900.0,785000.0,828500.0,877500.0
90018,95999,California,1075,NaN,NaN,NaN,NaN,NaN,372900.0,359300.0,...,723400.0,716800.0,821700.0,834000.0,855600.0,765300.0,767500.0,766500.0,768800.0,784300.0


In [5]:
california_df = california_df.dropna()
california_df.head()

,RegionID,StateName,SizeRank,2008-03,2008-04,2008-05,2008-06,2008-07,2008-08,2008-09,...,2019-05,2019-06,2019-07,2019-08,2019-09,2019-10,2019-11,2019-12,2020-01,2020-02
RegionName,,,,,,,,,,,,,,,,,,,,,
90002,95983,California,2748,292100.0,296700.0,268800.0,238900.0,203200.0,202100.0,172800.0,...,379500.0,385100.0,388900.0,381400.0,387800.0,400800.0,408800.0,393500.0,378600.0,387600.0
90016,95997,California,1060,462900.0,416000.0,347700.0,347700.0,337300.0,339800.0,320800.0,...,714100.0,737500.0,734900.0,733300.0,723000.0,760400.0,766900.0,785000.0,828500.0,877500.0
90025,96006,California,227,719700.0,670400.0,644700.0,638300.0,651500.0,660900.0,659000.0,...,944800.0,951000.0,952500.0,924600.0,983900.0,993000.0,1026000.0,960600.0,913400.0,921900.0
90032,96013,California,2254,355700.0,387900.0,364700.0,353900.0,338700.0,329900.0,349300.0,...,596100.0,591600.0,628400.0,627200.0,617800.0,607700.0,624800.0,642900.0,640900.0,628000.0
90039,96020,California,2758,665800.0,626500.0,599000.0,587000.0,548700.0,533100.0,584300.0,...,1143800.0,1069800.0,1051200.0,1100800.0,1133100.0,1099000.0,1100800.0,1040300.0,939000.0,935800.0


In [6]:
test =california_df.loc[:,'2008-03':'2008-12']
test.head()

,2008-03,2008-04,2008-05,2008-06,2008-07,2008-08,2008-09,2008-10,2008-11,2008-12
RegionName,,,,,,,,,,
90002,292100.0,296700.0,268800.0,238900.0,203200.0,202100.0,172800.0,167200.0,174300.0,171500.0
90016,462900.0,416000.0,347700.0,347700.0,337300.0,339800.0,320800.0,343900.0,341600.0,337500.0
90025,719700.0,670400.0,644700.0,638300.0,651500.0,660900.0,659000.0,640200.0,629600.0,589600.0
90032,355700.0,387900.0,364700.0,353900.0,338700.0,329900.0,349300.0,327600.0,314300.0,287400.0
90039,665800.0,626500.0,599000.0,587000.0,548700.0,533100.0,584300.0,597800.0,608100.0,521500.0


In [7]:
median09 = california_df.loc[:,'2009-01':'2009-12'].median(axis = 1, skipna = True)
median09

RegionName
90002    126300.0
90016    275300.0
90025    616700.0
90032    239550.0
90039    552300.0
           ...   
96002    189600.0
96003    216600.0
96022    181400.0
96150    307900.0
96161    469950.0
Length: 385, dtype: float64

In [8]:
median19 = california_df.loc[:,'2019-01':'2019-12'].median(axis = 1, skipna = True)
median19

RegionName
90002     390150.0
90016     741600.0
90025     947900.0
90032     616650.0
90039    1080600.0
           ...    
96002     268150.0
96003     292050.0
96022     268150.0
96150     454650.0
96161     641250.0
Length: 385, dtype: float64

In [9]:
new_df=pd.concat([median09, median19], axis=1)
new_name= ['Median Price 2009','Median Price 2019']
new_df.columns=new_name
new_df.head()

,Median Price 2009,Median Price 2019
RegionName,,
90002,126300.0,390150.0
90016,275300.0,741600.0
90025,616700.0,947900.0
90032,239550.0,616650.0
90039,552300.0,1080600.0


In [10]:
new_df['Inflated Median Price 2009'] = 1.1917 * new_df['Median Price 2009']
new_df.head()

,Median Price 2009,Median Price 2019,Inflated Median Price 2009
RegionName,,,
90002,126300.0,390150.0,150511.710
90016,275300.0,741600.0,328075.010
90025,616700.0,947900.0,734921.390
90032,239550.0,616650.0,285471.735
90039,552300.0,1080600.0,658175.910


In [11]:
new_df['Percent Change'] = (new_df['Median Price 2019'] - new_df['Inflated Median Price 2009'])/new_df['Inflated Median Price 2009']
new_df.head()

,Median Price 2009,Median Price 2019,Inflated Median Price 2009,Percent Change
RegionName,,,,
90002,126300.0,390150.0,150511.710,1.592157
90016,275300.0,741600.0,328075.010,1.260459
90025,616700.0,947900.0,734921.390,0.289798
90032,239550.0,616650.0,285471.735,1.160109
90039,552300.0,1080600.0,658175.910,0.641810


In [12]:
outcome_df = new_df.copy()

In [18]:
outcome = [] 
for value in outcome_df["Percent Change"]: 
    if value >= 0.80: 
        outcome.append(1)  
    else: 
        outcome.append(0) 
       
outcome_df["Outcome"] = outcome

In [14]:
# new_df = new_df[['Median Price 2009','Inflated Median Price 2009','Median Price 2019','Percent Change','Outcome']]
# new_df.head()

In [19]:
outcome_df.count()

Median Price 2009             385
Median Price 2019             385
Inflated Median Price 2009    385
Percent Change                385
Outcome                       385
dtype: int64

In [20]:
outcome_count = outcome_df.Outcome.value_counts()
outcome_count

0    321
1     64
Name: Outcome, dtype: int64

In [ ]:
# new_df.to_csv ('california_df.csv', index = 'RegionName', header=True)

In [ ]:
formatted_df = new_df.copy()

In [ ]:
formatted_df['Median Price 2009'] = formatted_df['Median Price 2009'].map("${:,.2f}".format)
formatted_df['Inflated Median Price 2009'] = formatted_df['Inflated Median Price 2009'].map("${:,.2f}".format)
formatted_df['Median Price 2019'] = formatted_df['Median Price 2019'].map("${:,.2f}".format)
formatted_df['Percent Change'] = formatted_df['Percent Change'].map("{:.2%}".format)
formatted_df.head()